In [2]:
#!/usr/bin/env python
from __future__ import print_function, division
# coding: utf-8

# In[1]:


import torch
import sys
import antialiased_cnns
# from torchvision import datasets, models, transforms

from PIL import Image
import torchvision

sys.path.append('/om5/user/smadan/training_scaffold_own/res/')
from models.models import get_model
from loader.loader import get_loader
import random
import pickle
import argparse
import wandb

# parser = argparse.ArgumentParser()
# parser.add_argument('--num_epochs', type = int, default = 50)
# parser.add_argument('--model_arch', type = str, default = 'resnet18')
# parser.add_argument('--batch_size', type = int, default = 100)
# parser.add_argument('--num_classes', type = int, default = 11)
# parser.add_argument('--base_lr', type = float, default = 0.001)
# parser.add_argument('--use_gpu', type = bool, default = True)
# parser.add_argument('--run_name', type = str)
# parser.add_argument('--pretrained', action='store_true')
# parser.add_argument('--image_size', type = int, default = 224)
# parser.add_argument('--dataset_name', type = str, required = True)
# parser.add_argument('--normalize', action='store_true')

# args = parser.parse_args()

# wandb_config = dict(vars(args))
wandb_config = {'num_epochs': 50, 'model_arch': 'resnet18', 'batch_size': 125, 'num_classes': 11, 'base_lr': 0.0003, 'use_gpu': True, 'run_name': 'resnet18_v7_normalized', 'pretrained': False, 'image_size': 224, 'dataset_name': 'image_train_v7_shapenet', 'normalize': True} 

# wandb.login()
# NUM_EPOCHS = args.num_epochs
# BATCH_SIZE = args.batch_size
# MODEL_ARCH = args.model_arch
# NUM_CLASSES = args.num_classes
# BASE_LR = args.base_lr
# USE_GPU = args.use_gpu
# SAVE_PATH = args.save_path
# LOG_FILE = args.log_file
# DATASET_NAME = args.dataset_name


# LOG_FILE_HANDLE = open(LOG_FILE, 'w')

loader = get_loader('multi_attribute_loader_file_list_shapenet')





In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections
from collections import OrderedDict
from tqdm.notebook import tqdm


input_img_size = wandb_config['image_size']

image_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#### Below was implemented for LeNet, leave commented ####
# image_transform = transforms.Compose([
#     transforms.Resize((input_img_size,input_img_size)),
#     transforms.Grayscale(),
#     transforms.ToTensor()
# #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

loader_new = get_loader('multi_attribute_loader_file_list_shapenet')
file_list_root = '/om5/user/smadan/dataset_lists_openmind'
att_path = '/om5/user/smadan/differentiable_graphics_ml/training_models/shapenet_id_to_class_num.p'
shuffles = {'train':True,'val':True,'test':False}

In [4]:
count=0

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.ToTensor()
    ]),
}

file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}
for phase in ['train','test']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,wandb_config['dataset_name'])
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform)
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=wandb_config['batch_size'], shuffle = shuffles[phase], num_workers=2,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])